# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

* import json dataset
```
mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json
```

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'local', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['estalishments', 'establishments']


In [30]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('641267b7dec6b2a566accc97'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [31]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [32]:
# Create a dictionary for the new restaurant data
# city, name
new_restaurant = {'BusinessName': 'Penang Flavours', 'AddressLine1': 'Greenwich'}

In [33]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [34]:
# Check that the new restaurant was inserted
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

{'AddressLine1': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 '_id': ObjectId('641501b8e80774e862cdb028')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [36]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType' : "Restaurant/Cafe/Canteen"}
results = establishments.find_one(query)
BusinessTypeID = result['BusinessTypeID']



3. Update the new restaurant with the `BusinessTypeID` you found.

In [42]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({'BusinessName': 'Penang Flavours'},
                           {'$set': 
                                {'BusinessTypeID': BusinessTypeID}
                           }
                           )



{'AddressLine1': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('641501b8e80774e862cdb028')}


In [43]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

{'AddressLine1': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('641501b8e80774e862cdb028')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [44]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': "Dover"}
print("Number of documents in result:", establishments.count_documents(query))

Number of documents in result: 994


In [47]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)


In [48]:
# Check if any remaining documents include Dover
results = establishments.find(query)
for result in results:
    print(result)

In [49]:
# Check that other documents remain with 'find_one'
establishments.find_one({})

{'_id': ObjectId('641267b7dec6b2a566accf7e'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': '4',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  '

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude


In [ ]:
# Change the data type from String to Decimal for latitude


In [ ]:
# Check that the coordinates are now numbers
